# Comparing Differences Across Pandas DataFrames

In [1]:
import numpy as np
import pandas as pd

Creating DataFrame with some fake data

In [2]:
df = pd.DataFrame({
    'record_id': [1, 2, 3, 5],
    'name': ['Alice', 'Ben', 'Caroline', 'Edgar'],
    'col1': [5.5, 1.0, 2.3, 4.4],
    'col2': [6.7, 1.1, 2.6, 4.7],
})

df2 = pd.DataFrame({
    'record_id': [1, 2, 3, 4],
    'name': ['Abba', 'Ben', 'Caroline', 'Donna'],
    'col1': [5.5, 1.7, 2.3, 6.5],
    'col2': [6.7, 1.4, 2.6, 6.3]
})

In [3]:
df

,record_id,name,col1,col2
0,1,Alice,5.5,6.7
1,2,Ben,1.0,1.1
2,3,Caroline,2.3,2.6
3,5,Edgar,4.4,4.7


In [4]:
df2

,record_id,name,col1,col2
0,1,Abba,5.5,6.7
1,2,Ben,1.7,1.4
2,3,Caroline,2.3,2.6
3,4,Donna,6.5,6.3


Joining columns together, and swapping levels to make comparisons easier. First level are the original column names, second level is the "first" data frame vs the "second" dataframe

In [5]:
df_all = pd.concat(
    [df.set_index('record_id'), df2.set_index('record_id')], 
    axis='columns',
    keys=['First', 'Second']
)

df_final = df_all.swaplevel(axis='columns')[df.columns[1:]]

In [6]:
df_final

name            col1         col2       
              First    Second First Second First Second
record_id                                              
1             Alice      Abba   5.5    5.5   6.7    6.7
2               Ben       Ben   1.0    1.7   1.1    1.4
3          Caroline  Caroline   2.3    2.3   2.6    2.6
4               NaN     Donna   NaN    6.5   NaN    6.3
5             Edgar       NaN   4.4    NaN   4.7    NaN

Intuitively, we can eyeball the differences above. In particular, in the example above: 
1. `record_id=1` the names are difference ("Alice" vs. "Abba").
2. `record_id=2` the names are the same, but both `col1` and `col2` are different
3. `record_id=3` there are no differences.
4. `record_id=4` exists in the first data frame but not in the second
5. `record_id=5` exists in the second data frame but not in the first

We can formally show the differences (example below). First, we visually highlight the differences. Second, we pull out a DataFrame that gives a boolean matrix if the values between the same columns in the two dataframes are different. From there we can generate arbitrary number of summary statistics

In [12]:
def highlight_diff(data, color='yellow'):
    """Highlight differences between columns given same index"""
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)



In [13]:
df_final.style.apply(highlight_diff, axis=None)

Highlighted differences above; could be useful for small dataframes to eyeball differences. However, we can more comprehensively summarize by using the actual boolean values for whether the columns are difference for the same `record_id`

In [14]:
other = df_final.xs('First', axis='columns', level=-1)
df_diffs = df_final.ne(other, level=0)
df_diffs

name          col1          col2       
           First Second  First Second  First Second
record_id                                          
1          False   True  False  False  False  False
2          False  False  False   True  False   True
3          False  False  False  False  False  False
4           True   True   True   True   True   True
5          False   True  False   True  False   True

For example, we can pull the records and how many columns are different for each record

In [15]:
df_diffs.xs('Second', axis=1, level=1, drop_level=False).sum(axis=1)

record_id
1    1
2    2
3    0
4    3
5    3
dtype: int64

We can then take this and summarize the distribution of different columns across all records

In [16]:
df_diffs.xs('Second', axis=1, level=1, drop_level=False).sum(axis=1).value_counts()

3    2
2    1
1    1
0    1
dtype: int64

In this case, there are 2 rows with 3 differences (these are the two rows where one record_id exists in one but completely not in the other), there is 1 row with 2 differences, 1 row with 1 difference, and 1 row with 0 differences.